In [1]:
from neopipe.pipeline import Pipeline

In [2]:
pipeline = Pipeline()

In [3]:
data = [
    {"name": f"User{i % 10}", "earning": i % 7, "day": i % 5} for i in range(25)
]

In [5]:
from typing import List, Dict, Any
from neopipe.result import Result, Ok, Err

In [6]:
@pipeline.task()
def find_unique_names(data: List[Dict[str, Any]]) -> Result[List[str], str]:
    try:
        unique_names = list(set(item['name'] for item in data))
        return Ok(unique_names)
    except Exception as e:
        return Err(f"Error finding unique names: {str(e)}")

In [11]:
@pipeline.task()
def sort_users(data: List[str]) -> Result[List[str], str]:
    try:
        sorted_data = sorted(data)
        return Ok(sorted_data)
    except Exception as e:
        return Err(f"Error sorting users: {str(e)}")

In [12]:
result = pipeline.run(initial_value=data)

INFO:root:Task find_unique_names succeeded on attempt 1
INFO:root:Task sort_users succeeded on attempt 1


In [13]:
if result.is_ok():
    print(result.value)

['User0', 'User1', 'User2', 'User3', 'User4', 'User5', 'User6', 'User7', 'User8', 'User9']
